# DeepFM

DeepFM can be seen as an improvement of WDL and FNN.Compared with WDL,DeepFM use FM instead of LR in the wide part and use concatenation of embedding vectors as the input of MLP in the deep part. Compared with FNN,the embedding vector of FM and input to MLP are same. And they do not need a FM pretrained vector to initialiaze,they are learned end2end.

<img src='https://deepctr-doc.readthedocs.io/en/latest/_images/DeepFM.png'>

In [1]:
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import DeepFM
from deepctr.utils import SingleFeat

In [2]:
data = pd.read_csv('./criteo_sample.txt')

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

Label Encoding for sparse features,and do simple Transformation for dense features

In [3]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

/home/user/.local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
DeepCTR version 0.4.0.post0 detected. Your version is 0.4.0.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.4.0.post0


count unique features for each sparse field,and record dense feature field name

In [4]:
sparse_feature_list = [SingleFeat(feat, data[feat].nunique())
                       for feat in sparse_features]
dense_feature_list = [SingleFeat(feat, 0,)
                      for feat in dense_features]

generate input data for model

In [5]:
train, test = train_test_split(data, test_size=0.2)
train_model_input = [train[feat.name].values for feat in sparse_feature_list] + \
                    [train[feat.name].values for feat in dense_feature_list]
test_model_input = [test[feat.name].values for feat in sparse_feature_list] + \
                   [test[feat.name].values for feat in dense_feature_list]

Define Model, train, predict and evaluate

In [6]:
model = DeepFM({"sparse": sparse_feature_list,
                "dense": dense_feature_list}, task='binary')
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'], )

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

Train on 128 samples, validate on 32 samples
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Epoch 1/10
 - 2s - loss: 1.2060 - binary_crossentropy: 1.2059 - val_loss: 1.1693 - val_binary_crossentropy: 1.1693
Epoch 2/10
 - 0s - loss: 1.1176 - binary_crossentropy: 1.1176 - val_loss: 1.1102 - val_binary_crossentropy: 1.1102
Epoch 3/10
 - 0s - loss: 1.0356 - binary_crossentropy: 1.0356 - val_loss: 1.0544 - val_binary_crossentropy: 1.0544
Epoch 4/10
 - 0s - loss: 0.9595 - binary_crossentropy: 0.9594 - val_loss: 1.0014 - val_binary_crossentropy: 1.0013
Epoch 5/10
 - 0s - loss: 0.8882 - binary_crossentropy: 0.8882 - val_loss: 0.9509 - val_binary_crossentropy: 0.9508
Epoch 6/10
 - 0s - loss: 0.8216 - binary_crossentropy: 0.8216 - val_loss: 0.9019 - val_binary_crossentropy: 0.9019
Epoch 7/10
 - 0s - loss: 0.7589 - binary_crossentropy: 0.7588 - val_loss: 0.8541 - val_binary_crossentropy: 0.8541
Epoch 8/10
 - 0s - loss: 0.6998 - binary_crossentropy: 0.6997 - val_loss: 0.8083 - val_binary_crossentropy: 0.8082
Epoch 9/10
 - 0s - loss: 0.6444 - binary_crossentropy: 0.6444 - val_loss: 0.7640

In [8]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [9]:
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

('test LogLoss', 1.0549)
('test AUC', 0.2857)


## And Now with hash

In [10]:
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import DeepFM
from deepctr.utils import SingleFeat

In [11]:
data = pd.read_csv('./criteo_sample.txt')

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

Do simple Transformation for dense features

In [12]:
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

Set hashing space for each sparse field,and record dense feature field name

In [13]:
sparse_feature_list = [SingleFeat(feat, 1000, hash_flag=True, dtype='string')  # since the input is string
                           for feat in sparse_features]
dense_feature_list = [SingleFeat(feat, 0, ) for feat in dense_features]

generate input data for model

In [14]:
train, test = train_test_split(data, test_size=0.2)
train_model_input = [train[feat.name].values for feat in sparse_feature_list] + \
                    [train[feat.name].values for feat in dense_feature_list]
test_model_input = [test[feat.name].values for feat in sparse_feature_list] + \
                   [test[feat.name].values for feat in dense_feature_list]

Define Model, train, predict and evaluate

In [15]:
model = DeepFM({"sparse": sparse_feature_list,
                "dense": dense_feature_list}, task='binary')
model.compile("adam", "binary_crossentropy",
              metrics=['binary_crossentropy'], )

In [16]:
history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

Train on 128 samples, validate on 32 samples
Epoch 1/10
 - 2s - loss: 0.8388 - binary_crossentropy: 0.8388 - val_loss: 0.7626 - val_binary_crossentropy: 0.7626
Epoch 2/10
 - 0s - loss: 0.7905 - binary_crossentropy: 0.7904 - val_loss: 0.7425 - val_binary_crossentropy: 0.7425
Epoch 3/10
 - 0s - loss: 0.7466 - binary_crossentropy: 0.7466 - val_loss: 0.7241 - val_binary_crossentropy: 0.7240
Epoch 4/10
 - 0s - loss: 0.7066 - binary_crossentropy: 0.7066 - val_loss: 0.7070 - val_binary_crossentropy: 0.7070
Epoch 5/10
 - 0s - loss: 0.6697 - binary_crossentropy: 0.6697 - val_loss: 0.6911 - val_binary_crossentropy: 0.6910
Epoch 6/10
 - 0s - loss: 0.6352 - binary_crossentropy: 0.6352 - val_loss: 0.6760 - val_binary_crossentropy: 0.6759
Epoch 7/10
 - 0s - loss: 0.6027 - binary_crossentropy: 0.6026 - val_loss: 0.6616 - val_binary_crossentropy: 0.6615
Epoch 8/10
 - 0s - loss: 0.5718 - binary_crossentropy: 0.5718 - val_loss: 0.6478 - val_binary_crossentropy: 0.6478
Epoch 9/10
 - 0s - loss: 0.5425 - b

In [17]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [18]:
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

('test LogLoss', 0.6435)
('test AUC', 0.528)
